In [2]:
import os
from pathlib import Path
import json
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from coco.coco import COCO
from itertools import chain, combinations
import glob
import cv2
from skimage.measure import label, regionprops

%matplotlib inline

In [20]:
# label_dir = '/data/shared/PartImageNet/'
label_dir = '/global/scratch/users/nabeel126/PartImageNet/'

split = 'test'
root = Path('/global/scratch/users/nabeel126/PartImageNet/PartBoxSegmentations')

In [21]:
CLASSES = {
    "Quadruped": 4,
    "Biped": 5,
    "Fish": 4,
    "Bird": 5,
    "Snake": 2,
    "Reptile": 4,
    "Car": 3,
    "Bicycle": 4,
    "Boat": 2,
    "Aeroplane": 5,
    "Bottle": 2,
}
CLASSES = dict(sorted(CLASSES.items()))
CLASSES

{'Aeroplane': 5,
 'Bicycle': 4,
 'Biped': 5,
 'Bird': 5,
 'Boat': 2,
 'Bottle': 2,
 'Car': 3,
 'Fish': 4,
 'Quadruped': 4,
 'Reptile': 4,
 'Snake': 2}

In [22]:
PATHS = {
    "train": (root / "train", root / "image_labels" / 'train.json', root / "annotations" / 'train.json'),
    "val": (root / "val", root / "image_labels" / 'val.json', root / "annotations" / 'val.json'),
    "test": (root / "test", root / "image_labels" / 'test.json', root / "annotations" / 'test.json' ),
}

os.makedirs(root / "train", exist_ok=True)
os.makedirs(root / "val", exist_ok=True)
os.makedirs(root / "test", exist_ok=True)
os.makedirs(root / "image_labels", exist_ok=True)
os.makedirs(root / "annotations", exist_ok=True)

In [23]:
categories_json = []

part_id = 0
categories_json.append(
    {'supercategory': 'background',
    'id': 0,
    'name': 'background'}
)
part_id += 1

for part_imagenet_class in CLASSES:
    for id in range(CLASSES[part_imagenet_class]):
        categories_json.append(
            {'supercategory': part_imagenet_class,
            'id': part_id,
            'name': f'{part_imagenet_class}_{part_id}'}
        )
        part_id += 1
categories_json

[{'supercategory': 'background', 'id': 0, 'name': 'background'},
 {'supercategory': 'Aeroplane', 'id': 1, 'name': 'Aeroplane_1'},
 {'supercategory': 'Aeroplane', 'id': 2, 'name': 'Aeroplane_2'},
 {'supercategory': 'Aeroplane', 'id': 3, 'name': 'Aeroplane_3'},
 {'supercategory': 'Aeroplane', 'id': 4, 'name': 'Aeroplane_4'},
 {'supercategory': 'Aeroplane', 'id': 5, 'name': 'Aeroplane_5'},
 {'supercategory': 'Bicycle', 'id': 6, 'name': 'Bicycle_6'},
 {'supercategory': 'Bicycle', 'id': 7, 'name': 'Bicycle_7'},
 {'supercategory': 'Bicycle', 'id': 8, 'name': 'Bicycle_8'},
 {'supercategory': 'Bicycle', 'id': 9, 'name': 'Bicycle_9'},
 {'supercategory': 'Biped', 'id': 10, 'name': 'Biped_10'},
 {'supercategory': 'Biped', 'id': 11, 'name': 'Biped_11'},
 {'supercategory': 'Biped', 'id': 12, 'name': 'Biped_12'},
 {'supercategory': 'Biped', 'id': 13, 'name': 'Biped_13'},
 {'supercategory': 'Biped', 'id': 14, 'name': 'Biped_14'},
 {'supercategory': 'Bird', 'id': 15, 'name': 'Bird_15'},
 {'supercatego

In [24]:
images_json = []
annotations_json = []

image_to_label = {}

global_image_id = 0
for class_label, part_imagenet_class in enumerate(CLASSES):
    print('part_imagenet_class', part_imagenet_class)
    # get filenames of all segmentation masks
    seg_label_path = os.path.join(label_dir, f'PartSegmentations/All/{split}/{part_imagenet_class}/*.tif')
    seg_filenames = glob.glob(seg_label_path)

    for filename in seg_filenames:
        # load segmentation
        im = Image.open(filename)
        width, height = im.size
        imarray = np.array(im)

        image_name = filename.split('/')[-1]
        image_id = image_name[:-4]    

        jpeg_image_name = image_id + '.JPEG'
        folder_id = image_id.split('_')[0]

        images_json.append({
            'file_name': f'{folder_id}/{jpeg_image_name}',
            'height': height,
            'width': width,
            'id': global_image_id
            })

        image_to_label[global_image_id] = class_label

        # get unique labels
        image_labels = list(np.unique(imarray))
        
        # remove background class
        if 0 in image_labels:
            image_labels.remove(0)

        assert len(image_labels) <= CLASSES[part_imagenet_class]

        for index, part_label in enumerate(image_labels):            
            # get sementation mask for object_id
            mask = (imarray == part_label) * 1
            mask = np.uint8(mask)

            lbl_0, num_regions = label(mask, return_num=True, connectivity=1) 
            props = regionprops(lbl_0)

            label_txt_lines = []
            for prop_index, prop in enumerate(props):
                min_row, min_col, max_row, max_col = prop.bbox

                bbox_width = max_col - min_col
                bbox_height = max_row - min_row

                cur_part_bbox = [min_col, min_row, bbox_width, bbox_height]

                annotations_json.append({
                    'image_id': global_image_id,
                    'bbox': cur_part_bbox,
                    'category_id': int(part_label),
                    'id': f'{image_id}_{index}_{prop_index}',
                    'area': bbox_width * bbox_height,
                    'is_crowd': 0
                })
        global_image_id += 1

part_imagenet_class Aeroplane
part_imagenet_class Bicycle
part_imagenet_class Biped
part_imagenet_class Bird
part_imagenet_class Boat
part_imagenet_class Bottle
part_imagenet_class Car
part_imagenet_class Fish
part_imagenet_class Quadruped
part_imagenet_class Reptile
part_imagenet_class Snake


In [25]:
# Serializing json
json_object = json.dumps(image_to_label, indent=4)
print(PATHS[split][1])
with open(PATHS[split][1], "w") as outfile:
    outfile.write(json_object)
    

/global/scratch/users/nabeel126/PartImageNet/PartBoxSegmentations/image_labels/test.json


In [26]:
part_imagenet_bbox_json = {
    'images': images_json,
    'annotations': annotations_json,
    'categories_json': categories_json
}

# Serializing json
json_object = json.dumps(part_imagenet_bbox_json, indent=4)
 
with open(PATHS[split][2], "w") as outfile:
    outfile.write(json_object)
    

In [6]:
# Opening JSON file
with open(PATHS[split][1], 'r') as openfile:
    json_object = json.load(openfile)

In [9]:
json_object[str(8526)]

8

In [23]:
from part_model.dataloader.part_imagenet import PartImageNetBBOXDataset

img_folder, class_label_file, ann_file = PATHS[split]
print(img_folder, class_label_file, ann_file)
# dataset = PartImageNetBBOXDataset(
#         img_folder, 
#         ann_file, 
#         transforms=None, 
#         return_masks=False,
#         aux_target_hacks=aux_target_hacks_list
#     )

ModuleNotFoundError: No module named 'panoptic_parts.utils.panoptic_parts'

In [6]:
from DINO.datasets.coco import CocoDetection
# dataset = CocoDetection(img_folder, ann_file, 
#         transforms=make_coco_transforms(image_set, fix_size=args.fix_size, strong_aug=strong_aug, args=args), 
#         return_masks=False,
#         aux_target_hacks=aux_target_hacks_list,
#     )

img_folder, ann_file = PATHS[split]
print(img_folder, ann_file)
dataset = CocoDetection(img_folder, ann_file, 
        transforms=None, 
        return_masks=False,
        aux_target_hacks=aux_target_hacks_list
    )

/home/nab_126/miniconda3/envs/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/data/shared/PartImageNet/PartBoxSegmentations/train /data/shared/PartImageNet/PartBoxSegmentations/annotations/train.json
loading annotations into memory...
Done (t=0.30s)
creating index...
index created!


In [7]:
for obj in json_object['annotations']:
    if obj['image_id'] == 0:
        # print(obj)
        print(obj['bbox'])

[28, 271, 178, 80]
[205, 248, 525, 101]
[301, 190, 151, 81]
[633, 268, 134, 22]
[248, 325, 368, 70]
[585, 151, 167, 106]
